<a href="https://colab.research.google.com/github/datastax/ragstack-ai/blob/main/examples/notebooks/nemo_guardrails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NVIDIA NeMo Guardrails

This notebooks demonstrates how to set up and use NVIDIA NeMo's Guardrails. [NVIDIA NeMo](https://www.nvidia.com/en-us/ai-data-science/generative-ai/nemo-framework/) is a cloud-native framework designed for AI models. [NeMo Guardrails](https://github.com/NVIDIA/NeMo-Guardrails) is a toolkit for adding programmable guardrails to LLM-based conversational applications. In this notebook, you will create an RAG-based application featuring a Retrieval rail that answers only queries relevant to the provided knowledge base. 

![NeMo Guardrails](resources/rails.png)

## Prerequisites

You will need a vector-enabled Astra database. This notebook uses OpenAI, though you can certainly use the NVIDIA models, as NeMo [supports all LLM providers supported by Langchain](https://github.com/NVIDIA/NeMo-Guardrails/blob/develop/docs/user_guides/configuration-guide.md#supported-llm-models).

* Create an [Astra vector database](https://docs.datastax.com/en/astra-serverless/docs/getting-started/create-db-choices.html).
* Create an [OpenAI account](https://openai.com/)
* Within your database, create an [Astra DB Access Token](https://docs.datastax.com/en/astra-serverless/docs/manage/org/manage-tokens.html) with Database Administrator permissions.
* Get your Astra DB Endpoint: 
  * `https://<ASTRA_DB_ID>-<ASTRA_DB_REGION>.apps.astra.datastax.com`

See the [Prerequisites](https://docs.datastax.com/en/ragstack/examples/prerequisites.html) page for more details.

## Setup

In [1]:
! pip install -qU ragstack-ai asyncio nemoguardrails


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import nest_asyncio

nest_asyncio.apply()

In [14]:
import os
from getpass import getpass

# Enter your settings for Astra DB and OpenAI:
os.environ["ASTRA_DB_API_ENDPOINT"] = input("Enter your Astra DB API Endpoint: ")
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass("Enter your Astra DB Token: ")
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

In [3]:
# Collections are where documents are stored. ex: test
collection = "nemo_guardrails_test"

## Create Guardrails

In [6]:
# The texts are pre-split into portions for ingestion
KNOWLEDGE_BASE = [
    "The year 2020 brought unprecedented challenges to the global financial economy, reshaping industries and markets in ways never before seen. From the onset of the COVID-19 pandemic to the profound shifts in consumer behavior, it was a year of resilience, adaptation, and lessons learned. In this blog post, we'll explore key events and statistics that defined the financial landscape of 2020, shedding light on the path forward.",  # noqa: E501
    "The COVID-19 pandemic led to a sharp economic downturn, with global GDP contracting by an estimated 3.5%. Major stock indices experienced historic declines, with the S&P 500 and Dow Jones Industrial Average falling by over 30% in March."  # noqa: E501
    "Governments worldwide responded with unprecedented fiscal and monetary stimulus packages. The total fiscal support provided amounted to over $12 trillion, including direct payments to individuals and loans/grants for businesses. The U.S. government alone passed stimulus packages totaling over $4 trillion, while the Federal Reserve expanded its balance sheet by more than $3 trillion."  # noqa: E501
    "Lockdowns and social distancing measures accelerated existing trends towards e-commerce and remote work. Global e-commerce sales surged by over 30% to $4.28 trillion, while the number of people working remotely doubled to over 40% of the global workforce.",  # noqa: E501
]

In [15]:
import os

from langchain_astradb import AstraDBVectorStore
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))
vstore = AstraDBVectorStore(
    collection_name=collection,
    embedding=embedding,
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
)
vstore.add_texts(KNOWLEDGE_BASE)
retriever = vstore.as_retriever()
print("Astra vector store configured")

Astra vector store configured


In [16]:
# Colang is a modeling language enabling the design of guardrails.
# In it, you define user and bot behaviors, as well as the flow of conversations.

COLANG_CONFIG = """
define user ask about economy
    "What was the economy like?"
    "What contributed to the growth or setback of the economy in 2020?"
    "What was the increase in e-commerce sales in 2020?"
    "How much did the S&P 500 fall in 2020?"

define flow answer economy question
    user ...
    $answer = execute rag()
    bot $answer

"""


def yaml_config(engine, model) -> str:
    return f"""
    models:
      - type: main
        engine: {engine}
        model: {model}
    """

In [23]:
from langchain.llms.base import BaseLLM
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from nemoguardrails import LLMRails
from nemoguardrails.actions.actions import ActionResult

TEMPLATE = """
Use the following context to answer the question. If you don't know the answer,
apologize and say that you don't know, do not make up an answer.

Context: {context}

Question: {question}

Answer:
"""


# Define the custom RAG Action
async def rag(context: dict, llm: BaseLLM) -> ActionResult:
    user_message = context.get("last_user_message")
    context_updates = {}

    # Use your pre-defined AstraDB Vector Store as the retriever
    relevant_documents = await retriever.aget_relevant_documents(user_message)
    relevant_chunks = "\n".join([chunk.page_content for chunk in relevant_documents])

    # Use a custom prompt template
    prompt_template = PromptTemplate.from_template(TEMPLATE)
    input_variables = {"question": user_message, "context": relevant_chunks}

    chain = prompt_template | llm | StrOutputParser()
    answer = await chain.ainvoke(input_variables)

    return ActionResult(return_value=answer, context_updates=context_updates)


def init(app: LLMRails) -> None:
    app.register_action(rag, "rag")

In [ ]:
from nemoguardrails import LLMRails, RailsConfig

engine = "openai"
model_name = "gpt-4"
yaml = yaml_config(engine, model_name)
config = RailsConfig.from_content(
    colang_content=COLANG_CONFIG,
    yaml_content=yaml,
)

rails = LLMRails(config)
init(rails)

In [24]:
# Your rails is now ready to answer questions about the provided knowledge base
response = rails.generate(
    messages=[{"role": "user", "content": "What was the economy like in 2020?"}]
)
print(response["content"])

Error in LoggingCallbackHandler.on_chat_model_start callback: TypeError('can only concatenate list (not "str") to list')


The economy in 2020 experienced unprecedented challenges due to the onset of the COVID-19 pandemic. The global GDP contracted by an estimated 3.5% and major stock indices like the S&P 500 and Dow Jones Industrial Average fell by over 30% in March 2020. Governments worldwide responded with unprecedented fiscal and monetary stimulus packages amounting to over $12 trillion. The U.S. government passed stimulus packages totaling over $4 trillion and the Federal Reserve expanded its balance sheet by more than $3 trillion. The pandemic also accelerated trends towards e-commerce and remote work, with global e-commerce sales surging by over 30% to $4.28 trillion and the number of people working remotely doubling to over 40% of the global workforce.


In [25]:
# Rails does not know about other subjects
response = rails.generate(
    messages=[
        {"role": "user", "content": "What was Michael Jordan's career high score?"}
    ]
)
print(response["content"])

Error in LoggingCallbackHandler.on_chat_model_start callback: TypeError('can only concatenate list (not "str") to list')


I'm sorry, but the information provided does not include details about Michael Jordan's career high score.
